In [1]:
import numpy as np
import tqdm
from tensorflow.contrib import slim

/home/david/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/david/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/david/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/david/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

In [2]:
import data as dataset

In [3]:
from dagan_networks_wgan import *

In [4]:
from utils.storage import build_experiment_folder, save_statistics
import utils.interpolations as interpolations
from utils.sampling import sample_generator, sample_two_dimensions_generator

In [5]:
data = dataset.OmniglotDAGANDataset(batch_size=32, last_training_class_index=900, reverse_channels=True,
                                    num_of_gpus=0, gen_batches=10)

In [6]:
experiment_name = 'omniglot_dagan_experiment_default'
saved_models_filepath, log_path, save_image_path = build_experiment_folder(experiment_name)

In [7]:
continue_from_epoch = -1

num_gpus = 0
batch_size = 32
gen_depth_per_layer = 3
discr_depth_per_layer = 3
z_dim = 100
num_generations = 64
dropout_rate_value = 0.5

reverse_channels = False

generator_layers = [16, 16, 32, 32]
discriminator_layers = [16, 16, 32, 32]

gen_inner_layers = [gen_depth_per_layer, gen_depth_per_layer, gen_depth_per_layer, gen_depth_per_layer]
discr_inner_layers = [discr_depth_per_layer, discr_depth_per_layer, discr_depth_per_layer,
                              discr_depth_per_layer]
generator_layer_padding = ["SAME", "SAME", "SAME", "SAME"]

image_height = data.image_height
image_width = data.image_width
image_channel = data.image_channel
input_x_i = tf.placeholder(tf.float32, [(num_gpus + 1), batch_size, image_height, image_width,
                                                     image_channel], 'inputs-1')
input_x_j = tf.placeholder(tf.float32, [(num_gpus + 1), batch_size, image_height, image_width,
                                                     image_channel], 'inputs-2-same-class')

z_input = tf.placeholder(tf.float32, [batch_size, z_dim], 'z-input')
training_phase = tf.placeholder(tf.bool, name='training-flag')
random_rotate = tf.placeholder(tf.bool, name='rotation-flag')
dropout_rate = tf.placeholder(tf.float32, name='dropout-prob')

dagan = DAGAN(batch_size=batch_size, input_x_i=input_x_i, input_x_j=input_x_j,
                      dropout_rate=dropout_rate, generator_layer_sizes=generator_layers,
                      generator_layer_padding=generator_layer_padding, num_channels=data.image_channel,
                      is_training=training_phase, augment=random_rotate,
                      discriminator_layer_sizes=discriminator_layers,
                      discr_inner_conv=discr_inner_layers,
                      gen_inner_conv=gen_inner_layers, num_gpus=num_gpus, z_dim=z_dim, z_inputs=z_input,
                      use_wide_connections=False)

In [8]:
summary, losses, graph_ops = dagan.init_train()

same_images = dagan.sample_same_images()

total_train_batches = int(data.training_data_size / (batch_size * (num_gpus + 1)))
    
total_gen_batches = int(data.generation_data_size / (batch_size * (num_gpus + 1)))

init = tf.global_variables_initializer()
spherical_interpolation = True
tensorboard_update_interval = int(total_train_batches/100/(num_gpus + 1))
total_epochs = 10

if continue_from_epoch == -1:
            save_statistics(log_path, ['epoch', 'total_d_train_loss_mean', 'total_d_val_loss_mean',
                                            'total_d_train_loss_std', 'total_d_val_loss_std',
                                            'total_g_train_loss_mean', 'total_g_val_loss_mean',
                                            'total_g_train_loss_std', 'total_g_val_loss_std'], create=True)




Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.dense instead.
Tensor("losses_0/generator/vector_expansion/Reshape:0", shape=(32, 2, 2, 8), dtype=float32, device=/device:CPU:0)
Tensor("losses_0/generator/vector_expansion/Reshape_1:0", shape=(32, 4, 4, 4), dtype=float32, device=/device:CPU:0)
Tensor("losses_0/generator/vector_expansion/Reshape_2:0", shape=(32, 7, 7, 2), dtype=float32, device=/device:CPU:0)
Tensor("losses_0/generator/vector_expansion/Reshape_3:0", shape=(32, 14, 14, 1), dtype=float32, device=/device:CPU:0)

Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.


generator_total_layers 50
generator_parameter_num 765881
Instructions for updating:
Use keras.layers.flatten instead.
discr layers 16
discriminator_param

In [9]:
"""RUN EXPERIMENT"""

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter("{}/train_logs/".format(log_path),
                                                      graph=tf.get_default_graph())
    validation_writer = tf.summary.FileWriter("{}/validation_logs/".format(log_path),
                                                           graph=tf.get_default_graph())
    train_saver = tf.train.Saver()
    val_saver = tf.train.Saver()

    start_from_epoch = 0
    if continue_from_epoch != -1:
        start_from_epoch = continue_from_epoch
        checkpoint = "{}train_saved_model_{}_{}.ckpt".format(saved_models_filepath, experiment_name, continue_from_epoch)
        variables_to_restore = []
        for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES):
            print(var)
            variables_to_restore.append(var)

        tf.logging.info('Fine-tuning from %s' % checkpoint)

        fine_tune = slim.assign_from_checkpoint_fn(
            checkpoint,
            variables_to_restore,
            ignore_missing_vars=True)
        fine_tune(sess)

    iter_done = 0
    disc_iter = 5
    gen_iter = 1
    best_d_val_loss = np.inf

    if spherical_interpolation:
        dim = int(np.sqrt(num_generations) * 2)
        z_2d_vectors = interpolations.create_mine_grid(rows=dim,
                                                            cols=dim,
                                                            dim=z_dim, space=3, anchors=None,
                                                            spherical=True, gaussian=True)
        z_vectors = interpolations.create_mine_grid(rows=1, cols=num_generations, dim=z_dim,
                                                         space=3, anchors=None, spherical=True, gaussian=True)
    else:
        z_vectors = np.random.normal(size=(num_generations, z_dim))
        z_2d_vectors = np.random.normal(size=(num_generations, z_dim))

    with tqdm.tqdm(total=total_epochs - start_from_epoch) as pbar_e:
        for e in range(start_from_epoch, total_epochs):
            
            print('epoch %d' % e)

            train_g_loss = []
            val_g_loss = []
            train_d_loss = []
            val_d_loss = []

            with tqdm.tqdm(total=total_train_batches) as pbar_train:
                for iter in range(total_train_batches):
                    
                    print('iter: ', iter)

                    cur_sample = 0

                    for n in range(disc_iter):
                        x_train_i, x_train_j = data.get_train_batch()
                        x_val_i, x_val_j = data.get_val_batch()

                        _, d_train_loss_value = sess.run(
                            [graph_ops["d_opt_op"], losses["d_losses"]],
                            feed_dict={input_x_i: x_train_i,
                                       input_x_j: x_train_j,
                                       dropout_rate: dropout_rate_value,
                                       training_phase: True, random_rotate: True})

                        d_val_loss_value = sess.run(
                            losses["d_losses"],
                            feed_dict={input_x_i: x_val_i,
                                       input_x_j: x_val_j,
                                       dropout_rate: dropout_rate_value,
                                       training_phase: False, random_rotate: False})

                        cur_sample += 1
                        train_d_loss.append(d_train_loss_value)
                        val_d_loss.append(d_val_loss_value)

                    for n in range(gen_iter):
                        x_train_i, x_train_j = data.get_train_batch()
                        x_val_i, x_val_j = data.get_val_batch()
                        _, g_train_loss_value, train_summaries = sess.run(
                            [graph_ops["g_opt_op"], losses["g_losses"],
                             summary],
                            feed_dict={input_x_i: x_train_i,
                                       input_x_j: x_train_j,
                                       dropout_rate: dropout_rate_value,
                                       training_phase: True, random_rotate: True})

                        g_val_loss_value, val_summaries = sess.run(
                            [losses["g_losses"], summary],
                            feed_dict={input_x_i: x_val_i,
                                       input_x_j: x_val_j,
                                       dropout_rate: dropout_rate_value,
                                       training_phase: False, random_rotate: False})

                        cur_sample += 1
                        train_g_loss.append(g_train_loss_value)
                        val_g_loss.append(g_val_loss_value)

                        if iter % (tensorboard_update_interval) == 0:
                            train_writer.add_summary(train_summaries, global_step=iter_done)
                            validation_writer.add_summary(val_summaries, global_step=iter_done)


                    iter_done = iter_done + 1
                    iter_out = "{}_train_d_loss: {}, train_g_loss: {}, " \
                               "val_d_loss: {}, val_g_loss: {}".format(iter_done,
                                                                       d_train_loss_value, g_train_loss_value,
                                                                       d_val_loss_value,
                                                                       g_val_loss_value)
                    pbar_train.set_description(iter_out)
                    pbar_train.update(1)

            total_d_train_loss_mean = np.mean(train_d_loss)
            total_d_train_loss_std = np.std(train_d_loss)
            total_g_train_loss_mean = np.mean(train_g_loss)
            total_g_train_loss_std = np.std(train_g_loss)

            print(
                "Epoch {}: d_train_loss_mean: {}, d_train_loss_std: {},"
                          "g_train_loss_mean: {}, g_train_loss_std: {}"
                .format(e, total_d_train_loss_mean,
                        total_d_train_loss_std,
                        total_g_train_loss_mean,
                        total_g_train_loss_std))

            total_d_val_loss_mean = np.mean(val_d_loss)
            total_d_val_loss_std = np.std(val_d_loss)
            total_g_val_loss_mean = np.mean(val_g_loss)
            total_g_val_loss_std = np.std(val_g_loss)

            print(
                "Epoch {}: d_val_loss_mean: {}, d_val_loss_std: {},"
                "g_val_loss_mean: {}, g_val_loss_std: {}, "
                    .format(e, total_d_val_loss_mean,
                            total_d_val_loss_std,
                            total_g_val_loss_mean,
                            total_g_val_loss_std))



            sample_generator(num_generations=num_generations, sess=sess, same_images=same_images,
                             inputs=x_train_i,
                             data=data, batch_size=batch_size, z_input=z_input,
                             file_name="{}/train_z_variations_{}_{}.png".format(save_image_path,
                                                                                experiment_name,
                                                                                e),
                             input_a=input_x_i, training_phase=training_phase,
                             z_vectors=z_vectors, dropout_rate=dropout_rate,
                             dropout_rate_value=dropout_rate_value)

            sample_two_dimensions_generator(sess=sess,
                                            same_images=same_images,
                                            inputs=x_train_i,
                                            data=data, batch_size=batch_size, z_input=z_input,
                                            file_name="{}/train_z_spherical_{}_{}".format(save_image_path,
                                                                                          experiment_name,
                                                                                          e),
                                            input_a=input_x_i, training_phase=training_phase,
                                            dropout_rate=dropout_rate,
                                            dropout_rate_value=dropout_rate_value,
                                            z_vectors=z_2d_vectors)

            with tqdm.tqdm(total=total_gen_batches) as pbar_samp:
                for i in range(total_gen_batches):
                    x_gen_a = data.get_gen_batch()
                    sample_generator(num_generations=num_generations, sess=sess,
                                     same_images=same_images,
                                     inputs=x_gen_a,
                                     data=data, batch_size=batch_size, z_input=z_input,
                                     file_name="{}/test_z_variations_{}_{}_{}.png".format(save_image_path,
                                                                                          experiment_name,
                                                                                          e, i),
                                     input_a=input_x_i, training_phase=training_phase,
                                     z_vectors=z_vectors, dropout_rate=dropout_rate,
                                     dropout_rate_value=dropout_rate_value)

                    sample_two_dimensions_generator(sess=sess,
                                                    same_images=same_images,
                                                    inputs=x_gen_a,
                                                    data=data, batch_size=batch_size,
                                                    z_input=z_input,
                                                    file_name="{}/val_z_spherical_{}_{}_{}".format(
                                                        save_image_path,
                                                        experiment_name,
                                                        e, i),
                                                    input_a=input_x_i,
                                                    training_phase=training_phase,
                                                    dropout_rate=dropout_rate,
                                                    dropout_rate_value=dropout_rate_value,
                                                    z_vectors=z_2d_vectors)

                    pbar_samp.update(1)

            train_save_path = train_saver.save(sess, "{}/train_saved_model_{}_{}.ckpt".format(
                saved_models_filepath,
                experiment_name, e))

            if total_d_val_loss_mean<best_d_val_loss:
                best_d_val_loss = total_d_val_loss_mean
                val_save_path = train_saver.save(sess, "{}/val_saved_model_{}_{}.ckpt".format(
                    saved_models_filepath,
                    experiment_name, e))
                print("Saved current best val model at", val_save_path)

            save_statistics(log_path, [e, total_d_train_loss_mean, total_d_val_loss_mean,
                                        total_d_train_loss_std, total_d_val_loss_std,
                                        total_g_train_loss_mean, total_g_val_loss_mean,
                                        total_g_train_loss_std, total_g_val_loss_std])

            pbar_e.update(1)

  0%|          | 0/562 [00:00<?, ?it/s]

epoch 0
iter:  0



1_train_d_loss: 4.220109939575195, train_g_loss: 0.4452425241470337, val_d_loss: 3.5693910121917725, val_g_loss: 0.6146197319030762:   0%|          | 0/562 [00:40<?, ?it/s]
1_train_d_loss: 4.220109939575195, train_g_loss: 0.4452425241470337, val_d_loss: 3.5693910121917725, val_g_loss: 0.6146197319030762:   0%|          | 1/562 [00:40<6:22:47, 40.94s/it]

iter:  1



2_train_d_loss: 3.0489706993103027, train_g_loss: 0.431510329246521, val_d_loss: 1.1116185188293457, val_g_loss: 0.9080435037612915:   0%|          | 1/562 [00:42<6:22:47, 40.94s/it]
2_train_d_loss: 3.0489706993103027, train_g_loss: 0.431510329246521, val_d_loss: 1.1116185188293457, val_g_loss: 0.9080435037612915:   0%|          | 2/562 [00:42<4:32:57, 29.25s/it]

...


5619_train_d_loss: -16.236242294311523, train_g_loss: 75.28478240966797, val_d_loss: -9.533233642578125, val_g_loss: 68.77951049804688: 100%|█████████▉| 560/562 [18:31<00:03,  2.00s/it] 
5619_train_d_loss: -16.236242294311523, train_g_loss: 75.28478240966797, val_d_loss: -9.533233642578125, val_g_loss: 68.77951049804688: 100%|█████████▉| 561/562 [18:31<00:02,  2.02s/it]

iter:  561



5620_train_d_loss: -8.325788497924805, train_g_loss: 99.26136016845703, val_d_loss: -5.14739990234375, val_g_loss: 90.79936218261719: 100%|█████████▉| 561/562 [18:33<00:02,  2.02s/it]  
5620_train_d_loss: -8.325788497924805, train_g_loss: 99.26136016845703, val_d_loss: -5.14739990234375, val_g_loss: 90.79936218261719: 100%|██████████| 562/562 [18:33<00:00,  1.98s/it]


Epoch 9: d_train_loss_mean: -8.095173835754395, d_train_loss_std: 16.000789642333984,g_train_loss_mean: 122.12019348144531, g_train_loss_std: 41.85170364379883
Epoch 9: d_val_loss_mean: -8.074599266052246, d_val_loss_std: 2.2901880741119385,g_val_loss_mean: 120.16908264160156, g_val_loss_std: 40.07436752319336, 


Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [1.9999998807907104, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.9999999403953552, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [5.999999642372131, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [12.039182782173157, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [4.620909243822098, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [3.999999761581421, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8